# 타이타닉 생존자 예측 (Titanic passengers Survival) with XGBoost & Grid Search

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 불러오기

In [2]:
train_df = pd.read_csv('../data/titanic_train.csv')
test_df = pd.read_csv('../data/titanic_test.csv')
(len(train_df), len(test_df))

(891, 418)

In [ ]:
train_len = len(train_df)
titanic_df = pd